In [1]:
from keras import layers
import keras_tuner as kt
import numpy as np
import os
import pretty_midi
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

/Users/roehr/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Data Collection
The analysis begins with a code block that traverses a directory structure to organize and collect file paths for MIDI files associated with four classical composers (Bach, Beethoven, Chopin, and Mozart). It iterates through each composer's folder, identifying .mid files, and stores their paths in a dictionary categorized by composer. Finally, it prints the count of MIDI files collected for each composer.

In [2]:
# Base directory containing the MIDI files organized by composer
base_directory = './midiclassics'

# Dictionary to store MIDI file paths categorized by composer
midi_files = {
    'Bach': [],
    'Beethoven': [],
    'Chopin': [],
    'Mozart': []
}

# Iterate through each composer folder and collect the paths of .mid files
for composer in midi_files.keys():
    composer_directory = os.path.join(base_directory, composer)
    for root, dirs, files in os.walk(composer_directory):
        for file in files:
            if file.endswith('.mid'):
                midi_files[composer].append(os.path.join(root, file))

# Print the number of MIDI files collected for each composer
for composer, files in midi_files.items():
    print(f"{composer}: {len(files)} files")

Bach: 925 files
Beethoven: 212 files
Chopin: 136 files
Mozart: 257 files


## Data Pre-processing
In the following step, a funcition is defined to preprocess MIDI files by converting them into a normalized piano roll format. It then applies this preprocessing function to MIDI files for each composer, storing the processed data in a dictionary categorized by composer. Finally, it prints the number of successfully processed pieces for each composer.

In [3]:
def preprocess_midi_file(midi_file):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_file)
        
        # Convert the MIDI file into a piano roll format
        piano_roll = midi_data.get_piano_roll(fs=100)
        
        # Normalize the piano roll by dividing by the maximum velocity
        piano_roll = piano_roll / np.max(piano_roll)
        
        return piano_roll
    except Exception as e:
        print(f"Error processing {midi_file}: {e}")
        return None

# Preprocess the MIDI files for each composer
preprocessed_data = {
    'Bach': [],
    'Beethoven': [],
    'Chopin': [],
    'Mozart': []
}

for composer, files in midi_files.items():
    for midi_file in files:
        processed_data = preprocess_midi_file(midi_file)
        if processed_data is not None:
            preprocessed_data[composer].append(processed_data)

# Print the shape of the processed data for each composer
for composer, data in preprocessed_data.items():
    print(f"{composer}: {len(data)} pieces processed")

/Users/roehr/myenv/lib/python3.9/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error processing ./midiclassics/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
Error processing ./midiclassics/Mozart/Piano Sonatas/Nueva carpeta/K281 Piano Sonata n03 3mov.mid: Could not decode key with 2 flats and mode 2
Bach: 925 pieces processed
Beethoven: 211 pieces processed
Chopin: 136 pieces processed
Mozart: 256 pieces processed


## Feature Extraction
The preprocssed MIDI files are inserted into a function that extracts musical features, including tempo, pitch class histogram (notes), and chroma features. It processes the MIDI files for each composer, extracting these features and storing them in a dictionary categorized by composer. Finally, it prints a summary of the extracted features, including the number of processed pieces and the shape of the resulting feature matrix for each composer.

In [4]:
def extract_features(midi_file):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_file)
        
        # Extract tempo (average across the entire piece)
        tempos = midi_data.get_tempo_changes()[1]
        avg_tempo = np.mean(tempos) if len(tempos) > 0 else 0
        
        # Extract notes (pitch class histogram)
        notes = np.zeros(128)
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                notes[note.pitch] += 1
        
        # Normalize the note counts to create a histogram
        notes /= np.sum(notes)
        
        # Extract chords (chroma feature, i.e., 12 pitch classes)
        chroma = midi_data.get_chroma()
        chroma = np.mean(chroma, axis=1)
        
        # Combine the features into a single vector
        features = np.concatenate([notes, chroma, [avg_tempo]])
        
        return features
    except Exception as e:
        print(f"Error extracting features from {midi_file}: {e}")
        return None

# Extract features from the preprocessed MIDI files
features_data = {
    'Bach': [],
    'Beethoven': [],
    'Chopin': [],
    'Mozart': []
}

for composer, files in midi_files.items():
    for midi_file in files:
        features = extract_features(midi_file)
        if features is not None:
            features_data[composer].append(features)

# Verify the feature extraction by printing a summary for each composer
for composer, data in features_data.items():
    if len(data) > 0:
        # Print the shape of the feature matrix for the composer
        print(f"{composer}: {len(data)} pieces with feature shape {np.array(data).shape}")
        
        # Optionally, print the first feature vector to inspect it
        print(f"Sample feature vector for {composer}: {data[0]}\n")
    else:
        print(f"No features extracted for {composer}")

Error extracting features from ./midiclassics/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
Error extracting features from ./midiclassics/Mozart/Piano Sonatas/Nueva carpeta/K281 Piano Sonata n03 3mov.mid: Could not decode key with 2 flats and mode 2
Bach: 925 pieces with feature shape (925, 141)
Sample feature vector for Bach: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.02354145e-03 0.00000000e+00 5.11770727e-03 0.00000000e+00
 1.53531218e-02 0.00000000e+00 1.53531218e-02 1.63766633e-02
 0.00000000e+00 1.94472876e-02 0.

## Model Building, Training, & Evalualtion
The extracted musical features and labels were placed into NumPy arrays and the encoded labels into categorical format. The data was then split into training and testing sets and reshaped for input into a combined CNN-LSTM model. The model was built with convolutional layers for feature extraction and LSTM layers for sequence learning, followed by fully connected layers for classification. After training the model on the data, it was evaluated on the test set and computed metrics such as accuracy, precision, and recall were calculated.

In [5]:
# Combine all data and labels into arrays for training
all_features = []
all_labels = []

for composer, data in features_data.items():
    all_features.extend(data)
    all_labels.extend([composer] * len(data))

# Convert to NumPy arrays
all_features = np.array(all_features)
all_labels = np.array(all_labels)

# Encode labels into categorical format
label_mapping = {composer: idx for idx, composer in enumerate(features_data.keys())}
y = np.array([label_mapping[label] for label in all_labels])
y = to_categorical(y, num_classes=len(features_data))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, y, test_size=0.2, random_state=42, stratify=y)

# Assume input_shape is (sequence_length, num_features)
sequence_length = X_train.shape[1]  # Number of features (e.g., 128 for notes + 12 for chroma + 1 for tempo)
num_features = 1  # Single time step per feature vector, hence 1

# Reshape the data for CNN input (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], sequence_length, num_features)
X_test = X_test.reshape(X_test.shape[0], sequence_length, num_features)

# Model building function
def build_model(input_shape, num_classes):
    model = Sequential()

    # CNN layers
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())

    # LSTM layers
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))

    # Fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Build the model
input_shape = (sequence_length, num_features)
num_classes = len(features_data)
model = build_model(input_shape, num_classes)

# Print the model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# Predict the classes for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy, precision, recall
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')

# Print the evaluation metrics
print(f"Test Loss: {test_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

/Users/roehr/myenv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 139, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 69, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 69, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 67, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 33, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 33, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 33, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 33, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 223,556 (873.27 KB)

 Trainable params: 223,172 (871.77 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.5585 - loss: 1.1611 - val_accuracy: 0.6046 - val_loss: 1.0321
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6190 - loss: 0.8686 - val_accuracy: 0.6209 - val_loss: 0.8899
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6821 - loss: 0.7693 - val_accuracy: 0.6111 - val_loss: 0.9066
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6864 - loss: 0.7547 - val_accuracy: 0.6961 - val_loss: 0.8710
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.7017 - loss: 0.7457 - val_accuracy: 0.7255 - val_loss: 0.6908
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.7639 - loss: 0.6470 - val_accuracy: 0.7124 - val_loss: 0.7780
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.7248 - loss: 0.6487 - val_accuracy: 0.7549 - val_loss: 0.6597
Epoch 8/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.7487 - loss: 0.6492 - val_accuracy: 0.7484 - v

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7176 - loss: 0.8728
Test accuracy: 73.53%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Test Loss: 0.7908
Accuracy: 0.7353
Precision: 0.7356
Recall: 0.7353


## Interpretation
The model achieved a test accuracy of 73.53%, indicating a moderate level of correct predictions on the unseen data. Despite improvements during training, the validation and test losses suggest some overfitting or challenges in generalizing to new data. The precision and recall values, both around 0.735, reflect balanced performance in detecting true positives across the classes.

## Model Optimization: Fine-tuning Hyperparameters
The neural network model was built using a combination of CNN and LSTM layers to classify musical compositions. Hyperparameters such as filter counts, kernel sizes, LSTM units, and dropout rates were tuned using Keras Tuner's Hyperband strategy. The data was split into training and testing sets, and the model was trained and validated, with the best hyperparameters selected based on validation accuracy. The tuned model was then evaluated on the test set, where metrics like accuracy, precision, and recall were calculated. Finally, the best hyperparameters and the model's performance metrics were reported.

In [9]:
def build_model(hp):
    model = keras.Sequential()
    
    # CNN layers
    model.add(layers.Conv1D(
        filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv1_kernel', values=[3, 5, 7]),
        activation='relu',
        input_shape=(141, 1)))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Conv1D(
        filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=64),
        kernel_size=hp.Choice('conv2_kernel', values=[3, 5, 7]),
        activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.BatchNormalization())
    
    # LSTM layers
    model.add(layers.LSTM(
        units=hp.Int('lstm1_units', min_value=64, max_value=256, step=64),
        return_sequences=True))
    model.add(layers.Dropout(hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.LSTM(
        units=hp.Int('lstm2_units', min_value=32, max_value=128, step=32)))
    model.add(layers.Dropout(hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)))
    
    # Dense layers
    model.add(layers.Dense(
        units=hp.Int('dense1_units', min_value=64, max_value=256, step=64),
        activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout3', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create a tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='my_dir',
    project_name='composer_classification'
)

# Perform the search
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Print the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

# Evaluate the best model
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy with best model: {test_accuracy * 100:.2f}%")

# Make predictions with the best model
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy, precision, recall
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')

# Print the evaluation metrics
print(f"Test Loss: {test_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Trial 90 Complete [00h 01m 25s]
val_accuracy: 0.7777777910232544

Best val_accuracy So Far: 0.8202614188194275
Total elapsed time: 00h 28m 11s
Best Hyperparameters:
conv1_filters: 64
conv1_kernel: 7
conv2_filters: 192
conv2_kernel: 7
lstm1_units: 64
dropout1: 0.1
lstm2_units: 64
dropout2: 0.2
dense1_units: 192
dropout3: 0.2
learning_rate: 0.0002502739699524452
tuner/epochs: 50
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8075 - loss: 0.6500  
Test accuracy with best model: 82.03%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Test Loss: 0.6274
Accuracy: 0.8203
Precision: 0.8153
Recall: 0.8203


## Final Intrepretation
The hyperparameter tuning yielded a model with a validation accuracy of approximately 82%, which was the best performance achieved during the trials. The final model, using the best hyperparameters, achieved a test accuracy of 82.03%, indicating good generalization to unseen data. The model also shows a precision of 81.53% and a recall of 82.03%, suggesting that it performs well in correctly classifying the musical compositions. The relatively low test loss of 0.6274 further supports the model's effectiveness in this classification task.